In [ ]:
########## Credit ##########
''' @author: RemiLoka '''

In [ ]:
########## Lines to change ##########

pathToRepository = "/Users/remi/Desktop/ContenuStage/Smart-TL-Management"

sumoBinary = "/Users/remi/sumo/bin/sumo-gui"
sumoCmd = [sumoBinary,'-c','data/one_run/m/osm.sumocfg','-n', 'data/one_run/m/osm.net.xml']

########## Number of iterations ##########
count = 2000        # maximum = 960240

########## Time to launch of the CH algorithm ##########
start = 10          

In [ ]:
########## Import ##########

import traci
import sumolib
import time

import Initialisation
import CH
import PredictPos
import ModifyFile

In [ ]:
########## Initialisation ##########

listTime = []
listNbVeh = []
allCar = []

f_memory = open("memorybasics.csv", "w")
f_memory.write('Information,x,y' + "\n")
f_memory.close()

f_time = open("time.csv", "w")
f_time.write('Nb car,time' + "\n")
f_time.close()

f_rsu = open("rsu.csv", "w")
f_rsu.write('time,vehicle,previous,future' + "\n")
f_rsu.close()

f_CH = open("CH.csv", "w")
f_CH.write('lenVeh,CH,CHbck,CM' + "\n")
f_CH.close()

time_start = time.time()
traci.start(sumoCmd)

########## Position of Traffic Light ##########

net = sumolib.net.readNet('data/one_run/m/osm.net.xml')

traffic_pos = Initialisation.FindTL(net)

########## Selection of the lanes and edges ##########

[TotalBigEdge, TotalListLane, TotalEdge, TotalLane] = Initialisation.ImportantLanes()

########## Some informations ##########

print('number of traffic lighters: ', len(traffic_pos))
print('edge before: ', len(TotalEdge), 'edge after: ', len(TotalBigEdge))
print('lane before: ', len(TotalLane), 'lane after: ', len(TotalListLane))

In [ ]:
########## Launch of the simulation ##########

i = 0
while traci.simulation.getTime() <= count:

########## Simulation Step ##########

    traci.simulationStep()
    current_time = traci.simulation.getTime()

    action= i%9

########## List Vehicle ##########

    [listVehID, lenListVeh] = Initialisation.ListVehicles(TotalBigEdge, TotalListLane)

########## CH ##########

    if i == start:
        listNeighbor = CH.detectNeighbourg(listVehID)
        [listRole, listIDassigned, listRank]= CH.roleCar(listVehID,listNeighbor)
        qty = CH.assignColor(listIDassigned, listRole)
    if i > start:
        time0 = time.time()
        listNeighbor = CH.detectNeighbourg(listVehID)
        [ListRole, listIDassigned, ListRank] = CH.maintainCH(listVehID, listIDassigned, listRank, listRole, listNeighbor)
        time1 = time.time()
        qty = CH.assignColor(listIDassigned, listRole)

        f_CH = open("CH.csv", "a")
        f_CH.write(str(lenListVeh)+","+str(qty[0]) + "," + str(qty[1]) +","+ str(qty[2]) + "\n")
        f_CH.close

        f_time = open("time.csv", "a")
        f_duration = time1 - time0
        f_time.write(str(lenListVeh) + "," + str(f_duration) + "\n")
        f_time.close

########## Prediction position ##########

    timeSimu = traci.simulation.getTime()
    memory_time = '\ntime: %d \n' %timeSimu
    f_memory = open("memorybasics.csv", "a")
    f_memory.write(memory_time + "\n")

    f_rsu = open("rsu.csv", "a")
    RSU_time = '%d,' %timeSimu

    for vehicle in listVehID:
        if vehicle in allCar:
            pass
        else:
            allCar.append(vehicle)

        # Position
        position = traci.vehicle.getPosition(vehicle)
        futurPos = PredictPos.FuturPos(vehicle)
        memory_vehi = 'vehicle: %s' %vehicle
        memory_pos = 'Current Position:, %f,%f' %(position[0],position[1])
        memory_fut = 'Predict Position:, %f,%f' %(futurPos[0],futurPos[1])
        f_memory.write(memory_vehi + "\n" + memory_pos + "\n" + memory_fut + "\n")

        # RSU
        informationRSU = PredictPos.InformedCar(vehicle, traffic_pos)
        RSU_veh = '%s,' %vehicle
        RSU_previous = '%s,' %str(informationRSU[1])
        RSU_futur = '%s,' %str(informationRSU[0])
        f_rsu.write(RSU_time + RSU_veh + RSU_previous + RSU_futur+'\n')

    f_memory.close()
    f_rsu.close()

    i = i + 1 

########## End simulation ##########

time_end = time.time()
print ("traci time elapsed: {}".format(time_end-time_start))

In [ ]:
########## Analyse ##########

ModifyFile.traceTimeCurve()
ModifyFile.curveCH()

for vehicle in allCar:
    ModifyFile.calculationError(vehicle)

ModifyFile.calculationTot()

traci.close(True)